# SaaS Revenue & Expense Forecasting

Welcome! This notebook walks you through:

1. Loading a synthetic SaaS dataset
2. Exploring key metrics (MRR, churn, ARPA)
3. Forecasting Revenue (Holt-Winters), COGS, and OpEx
4. Building Base/Upside/Downside scenarios
5. Exporting results for Power BI and a report

> **Tip:** Run each code cell in order. If a cell errors, re-run it after correcting the issue.

## 0. Setup & Imports

In [ ]:
# If you don't have these packages, install them from a terminal:
# pip install pandas numpy matplotlib statsmodels scikit-learn

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Settings
plt.rcParams['figure.figsize'] = (10, 5)
pd.options.display.float_format = '{:,.2f}'.format

BASE_DIR = os.path.dirname(os.getcwd()) if os.getcwd().endswith('notebook') else os.getcwd()
DATA_PATH = os.path.join(BASE_DIR, 'data', 'saas_financials.csv')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

print('Using data file:', DATA_PATH)
print('Outputs will be saved to:', OUTPUT_DIR)

## 1. Load the Data

In [ ]:
df = pd.read_csv(DATA_PATH, parse_dates=['date'])
df = df.sort_values('date')
df.head()

## 2. Quick Exploration
Let's look at revenue, ARPA, and customers over time.

In [ ]:
# Plot Revenue
plt.figure()
plt.plot(df['date'], df['revenue'], label='Revenue')
plt.title('Revenue Over Time')
plt.xlabel('Date'); plt.ylabel('USD')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '01_revenue_over_time.png'))
plt.show()

# Plot Customers
plt.figure()
plt.plot(df['date'], df['customers_eop'], label='Customers (EOP)')
plt.title('Customers (End of Period)')
plt.xlabel('Date'); plt.ylabel('Count')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '02_customers_over_time.png'))
plt.show()

# Plot ARPA
plt.figure()
plt.plot(df['date'], df['arpa'], label='ARPA')
plt.title('Average Revenue per Account (ARPA)')
plt.xlabel('Date'); plt.ylabel('USD per account per month')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '03_arpa_over_time.png'))
plt.show()

## 3. Feature Engineering
Create helpful ratios and margins for KPI tracking.

In [ ]:
df['gross_margin'] = (df['revenue'] - df['cogs']) / df['revenue']
df['operating_margin'] = df['operating_income'] / df['revenue']
df['mrr_growth'] = df['mrr'].pct_change()
df['net_new_customers'] = df['new_customers'] - df['churned_customers']
df[['date','revenue','gross_margin','operating_margin','net_new_customers']].head()

## 4. Forecasting Revenue (Holt-Winters)
We'll use Exponential Smoothing (a standard time-series method).

In [ ]:
# Split train/test (last 6 months as test)
train = df.iloc[:-6].copy()
test = df.iloc[-6:].copy()

# Fit Holt-Winters with trend and seasonality (12-month seasonality)
y = train['revenue'].values
model = ExponentialSmoothing(train['revenue'],
                             trend='add',
                             seasonal='add',
                             seasonal_periods=12).fit()

# In-sample and out-of-sample predictions
train_pred = model.fittedvalues
test_pred = model.forecast(steps=len(test))

# Evaluate
mae = mean_absolute_error(test['revenue'], test_pred)
print(f'MAE on holdout: {mae:,.0f}')

# Plot
plt.figure()
plt.plot(train['date'], train['revenue'], label='Train')
plt.plot(test['date'], test['revenue'], label='Test')
plt.plot(train['date'], train_pred, label='Fitted')
plt.plot(test['date'], test_pred, label='Forecast')
plt.title('Revenue: Holt-Winters Forecast')
plt.xlabel('Date'); plt.ylabel('USD')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '04_revenue_forecast_hw.png'))
plt.show()

# Save base forecast for revenue
base_forecast_revenue = pd.DataFrame({
    'date': test['date'].values,
    'revenue_forecast_base': test_pred.values
})

## 5. Forecasting COGS and OpEx
- **COGS**: model as % of Revenue (trend + noise)
- **OpEx**: simple linear model against time and revenue.

In [ ]:
# COGS % model: regress COGS as a function of Revenue
X = train[['revenue']].values
y_cogs = train['cogs'].values
lin_cogs = LinearRegression().fit(X, y_cogs)

# Predict COGS on test revenue forecast (use forecasted revenue)
test_cogs_pred = lin_cogs.predict(test_pred.values.reshape(-1, 1))

# OpEx model: regress OpEx on time index and revenue
train = train.copy()
train['t'] = np.arange(len(train))
test = test.copy()
test['t'] = np.arange(len(train), len(train) + len(test))

X_opex = np.column_stack([train['t'].values, train['revenue'].values])
y_opex = train['opex'].values
lin_opex = LinearRegression().fit(X_opex, y_opex)

X_opex_test = np.column_stack([test['t'].values, test_pred.values])
test_opex_pred = lin_opex.predict(X_opex_test)

# Operating Income = Gross Profit - OpEx
test_gross_profit_pred = test_pred - test_cogs_pred
test_operating_income_pred = test_gross_profit_pred - test_opex_pred

# Combine into DataFrame
forecast_df = pd.DataFrame({
    'date': test['date'].values,
    'revenue_base': test_pred,
    'cogs_base': test_cogs_pred,
    'gross_profit_base': test_gross_profit_pred,
    'opex_base': test_opex_pred,
    'operating_income_base': test_operating_income_pred
})

forecast_df.head()

## 6. Scenario Analysis
We define simple knobs:
- `growth_multiplier` (affects revenue)
- `churn_delta` (affects net customers, approximated here by scaling revenue)
- `opex_change` (percentage change in OpEx from base)

These are **illustrative**—in a production model you would model customers and churn directly.

In [ ]:
def apply_scenarios(base_revenue, base_cogs, base_opex, 
                    growth_multiplier=1.0, churn_delta=0.0, opex_change=0.0):
    # Approximate churn impact by scaling revenue (negative churn_delta reduces revenue)
    revenue = base_revenue * growth_multiplier * (1 + churn_delta)
    cogs = base_cogs * (revenue / base_revenue)  # scale with revenue
    opex = base_opex * (1 + opex_change)
    gp = revenue - cogs
    op_inc = gp - opex
    return revenue, cogs, gp, opex, op_inc

# Build three scenarios
scenarios = {
    'base': {'growth_multiplier': 1.00, 'churn_delta': 0.00, 'opex_change': 0.00},
    'upside': {'growth_multiplier': 1.06, 'churn_delta': 0.02, 'opex_change': -0.03},
    'downside': {'growth_multiplier': 0.94, 'churn_delta': -0.03, 'opex_change': 0.02},
}

scen_frames = []
for name, params in scenarios.items():
    rev, cogs, gp, opex, opi = apply_scenarios(
        forecast_df['revenue_base'].values,
        forecast_df['cogs_base'].values,
        forecast_df['opex_base'].values,
        **params
    )
    scen = pd.DataFrame({
        'date': forecast_df['date'].values,
        'scenario': name,
        'revenue': rev,
        'cogs': cogs,
        'gross_profit': gp,
        'opex': opex,
        'operating_income': opi
    })
    scen_frames.append(scen)

scenarios_df = pd.concat(scen_frames, ignore_index=True)

# Save forecast/scenario results
forecast_results_path = os.path.join(OUTPUT_DIR, 'forecast_results.csv')
scenarios_df.to_csv(forecast_results_path, index=False)
print('Saved:', forecast_results_path)

# Plot each scenario revenue
for scen in scenarios_df['scenario'].unique():
    sub = scenarios_df[scenarios_df['scenario'] == scen]
    plt.figure()
    plt.plot(df.iloc[-6:]['date'], sub['revenue'].values, label=f'Revenue ({scen})')
    plt.title(f'Revenue Forecast - {scen.capitalize()} Scenario')
    plt.xlabel('Date'); plt.ylabel('USD')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f'05_revenue_{scen}.png'))
    plt.show()

## 7. Cash Runway (Simple)
We project cash using the last known cash balance and forecasted operating income as proxy for cash flow (no financing).

In [ ]:
last_cash = df.iloc[-7]['cash_balance']  # cash before forecast window begins
proj = scenarios_df.copy()
proj['cash_flow'] = proj['operating_income']  # proxy
proj['cash_balance'] = np.nan

results = []
for scen in proj['scenario'].unique():
    sub = proj[proj['scenario'] == scen].copy()
    cash = last_cash
    balances = []
    for cf in sub['cash_flow'].values:
        cash += cf
        balances.append(cash)
    sub['cash_balance'] = balances
    results.append(sub)

cash_df = pd.concat(results, ignore_index=True)

# Save
cash_results_path = os.path.join(OUTPUT_DIR, 'cash_results.csv')
cash_df.to_csv(cash_results_path, index=False)
print('Saved:', cash_results_path)

# Runway metric: months until cash < 0 (per scenario)
runway = {}
for scen in cash_df['scenario'].unique():
    sub = cash_df[cash_df['scenario'] == scen]
    neg = np.where(sub['cash_balance'].values < 0)[0]
    runway[scen] = int(neg[0] + 1) if len(neg) else len(sub)

runway

## 8. Create a Scenario Summary Table

In [ ]:
summary = (
    scenarios_df.groupby('scenario')
    .agg(revenue=('revenue', 'sum'),
         gross_profit=('gross_profit', 'sum'),
         opex=('opex', 'sum'),
         operating_income=('operating_income', 'sum'))
    .reset_index()
)

summary['gross_margin_pct'] = summary['gross_profit'] / summary['revenue']
summary['operating_margin_pct'] = summary['operating_income'] / summary['revenue']

# Add runway months from prior step
summary['runway_months'] = summary['scenario'].map(lambda s: np.nan)
for scen, months in locals().get('runway', {}).items():
    summary.loc[summary['scenario'] == scen, 'runway_months'] = months

summary_path = os.path.join(OUTPUT_DIR, 'scenario_summary.csv')
summary.to_csv(summary_path, index=False)
print('Saved:', summary_path)
summary

## 9. Power BI Guide (Step-by-Step)
Follow these steps inside **Power BI Desktop** after running this notebook:


1. **Get Data → Text/CSV** and import:
   - `data/saas_financials.csv`
   - `output/forecast_results.csv`
   - `output/cash_results.csv`
   - `output/scenario_summary.csv`
2. **Model View**: create relationships on `date` if needed (1:* from `saas_financials` to the others).
3. **Report View**: add visuals
   - Line chart: `date` on X, `revenue` (from `saas_financials`) and `revenue` (from `forecast_results`, filtered by scenario = base) as Values.
   - Clustered column chart: OpEx by function — if using historical only, add `rd_expense`, `sm_expense`, `ga_expense` as separate series.
   - Card visuals: Gross Margin %, Operating Margin %, Runway Months (from `scenario_summary` filtered by scenario).
4. **Slicers**:
   - Scenario slicer using the `scenario` column from `forecast_results` for comparing scenarios.
5. **Basic DAX (optional)**:
   - `Gross Margin % := DIVIDE(SUM(gross_profit), SUM(revenue))`
   - `Operating Margin % := DIVIDE(SUM(operating_income), SUM(revenue))`
6. **Publish** (optional): Save as `.pbix` and include it in your GitHub repo.


## 10. Export a One-Page Summary (Markdown)
This creates a quick summary you can paste into your report.

In [ ]:
latest_hist = df.iloc[-1]
base_sum = summary[summary['scenario'] == 'base'].iloc[0]

summary_md = f"""# One-Page Summary

**Latest Actuals (as of {latest_hist['date'].date()}):**
- Revenue (month): ${latest_hist['revenue']:,.0f}
- Gross Margin: {latest_hist['gross_margin']*100:.1f}%
- Operating Margin: {latest_hist['operating_margin']*100:.1f}%
- Cash Balance: ${latest_hist['cash_balance']:,.0f}

**Forecast (Base Scenario, next {len(forecast_df)} months):**
- Revenue (sum): ${base_sum['revenue']:,.0f}
- Gross Margin % (avg): {base_sum['gross_margin_pct']*100:.1f}%
- Operating Margin % (avg): {base_sum['operating_margin_pct']*100:.1f}%
- Runway (months): {int(base_sum['runway_months'])}

**Notes:**
- Revenue forecast via Holt-Winters.
- COGS and OpEx via simple driver models.
- Adjust scenarios in `scenarios` dict to test sensitivities.
"""

summary_file = os.path.join(OUTPUT_DIR, 'executive_summary.md')
with open(summary_file, 'w', encoding='utf-8') as f:
    f.write(summary_md)

print('Saved:', summary_file)